In [ ]:
from keras.layers import Dense, Input
from keras.models import Model
from dca.layers import ColwiseMultLayer


def build(adata, hid_size):
    input_size = adata.n_vars
    output_size = input_size
    hid_size = hid_size
    extra_models = {}

    input_layer = Input(shape=(input_size,), name='count')
    sf_layer = Input(shape=(1,), name='size_factors')

    decoder_output = Dense(hid_size, activation=None, name='center')(input_layer)
    mean = Dense(output_size, name='mean')(decoder_output)
    output = ColwiseMultLayer([mean, sf_layer])

    extra_models['mean_norm'] = Model(inputs=input_layer, outputs=mean)
    extra_models['decoded'] = Model(inputs=input_layer, outputs=decoder_output)
    
    model = Model(inputs=[input_layer, sf_layer], outputs=output)
 
    encoder = Model(inputs=model.input,
                    outputs=model.get_layer('center').output)


    print('dca: Calculating low dimensional representations...')

    adata.obsm['X_dca'] = encoder.predict({'count': adata.X,
                                           'size_factors': adata.obs.size_factors})        
    print('dca: Calculating reconstructions...')

    adata.X = model.predict({'count': adata.X,
                             'size_factors': adata.obs.size_factors})
    
    return adata

adata = build(adata)